<a href="https://colab.research.google.com/github/JacopoMangiavacchi/Swift-TensorFlow-Sample-Notebooks/blob/master/FizzBuzz_Swift_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import Foundation
import TensorFlow

In [0]:
func bitEncode(_ n: UInt, bits: Int = 10) -> [Float] {
    var array = [Float](repeating: 0, count: bits)
    for i in 0..<bits {
        array[bits - 1 - i] = Float(n >> i & 1)
    }
    return array
}

func labelEncode(_ n: UInt) -> [Float] {
    guard n > 0 else { return [0, 0, 0, 1] }
    var array: [Float] = [0, 0, 0, 0]

    if n % 15 == 0 {
        array[0] = 1
    }
    else if n % 3 == 0 {
        array[1] = 1
    }
    else if n % 5 == 0 {
        array[2] = 1
    }
    else {
        array[3] = 1
    }

    return array
}

In [0]:
let start = 101
let end = 1024
var X = [[Float]]()
var Y = [[Float]]()

for i in start..<end {
    X.append(bitEncode(UInt(i)))
    Y.append(labelEncode(UInt(i)))
}

In [0]:
struct FizzBuzz: Layer {
    var l1: Dense<Float>
    var l2: Dense<Float>
    var l3: Dense<Float>


    init(variables: Int = 1) {
        l1 = Dense<Float>(inputSize: variables, outputSize: 64, activation: relu)
        l2 = Dense<Float>(inputSize: 64, outputSize: 128, activation: relu)
        l3 = Dense<Float>(inputSize: 128, outputSize: 4, activation: softmax) // {$0}
    }
    
    @differentiable(wrt: (self, input))
    func applied(to input: Tensor<Float>) -> Tensor<Float> {
        var output = l1.applied(to: input)
        output = l2.applied(to: output)
        return l3.applied(to: output)

    }
}

In [5]:
let x = Tensor<Float>(shape: [Int32(end - start), 10], scalars: Array(X.joined()))
let y = Tensor<Float>(shape: [Int32(end - start), 4], scalars: Array(Y.joined()))

print(x.shape)
print(y.shape)

TensorShape(dimensions: [923, 10])
TensorShape(dimensions: [923, 4])


In [0]:
let optimizer = RMSProp<FizzBuzz, Float>(learningRate: 0.003)
var model = FizzBuzz(variables: 10)


for epoch in 1...10000 {
    let (cost, 𝛁model) = model.valueWithGradient { m -> Tensor<Float> in
        let ŷ = m.applied(to: x)
        //return meanSquaredError(predicted: ŷ, expected: y)
        return softmaxCrossEntropy(logits: ŷ, labels: y)
    }
    optimizer.update(&model.allDifferentiableVariables, along: 𝛁model)
  
    if epoch % 1000 == 0 {
        print("Epoch: \(epoch) Cost: \(cost)")
    }
}

Epoch: 1000 Cost: 0.93507534
Epoch: 2000 Cost: 0.87316465
Epoch: 3000 Cost: 0.8468716
Epoch: 4000 Cost: 0.8465544
Epoch: 5000 Cost: 0.84636366
Epoch: 6000 Cost: 0.84637314
Epoch: 7000 Cost: 0.8395175
Epoch: 8000 Cost: 0.83951527
Epoch: 9000 Cost: 0.83099586


In [0]:
var X_test = [[Float]]()
var Y_true = [[Float]]()

for i in 0..<100 {
    X_test.append(bitEncode(UInt(i)))
    Y_true.append(labelEncode(UInt(i)))
}

let xTest = Tensor<Float>(shape: [100, 10], scalars: Array(X_test.joined()))
let yTest = model.applied(to: xTest)

In [0]:
let yMax = yTest.argmax(squeezingAxis: 1).array
let yArray = Array(yTest.array)
let description = ["FizzBuzz", "Fizz    ", "Buzz    ", "        "]

for i in 0..<100 {
    print(i, description[Int(yMax[i].description)!], Y_true[i], yArray[i])
}